# 基础理论部分
0. Can you come up out 3 sceneraies which use AI methods?

Ans: {自动驾驶、智能家居、各类机器人}

1. How do we use Github; Why do we use Jupyter and Pycharm;

Ans: {github: 1. git add .;  2. git commit -m '123'; 3. git push origin your_branch 4.git pull oriign your_branch; 5 git clone git_address

jupyter: 使用方便，简单直观可视化效果好。
pyharm：强大的IDE, 貌似绝大部分的python开发者都用pycharm。}

2. What's the Probability Model?

Ans: 概率模型是指定随机变量出现的不确定性的概率。通过样本发生的概率判断其是否发生或产生相对正确的结果

3. Can you came up with some sceneraies at which we could use Probability Model?

Ans: 车牌摇号、文字纠错、谷歌搜索错误单词匹配、文本分类(垃圾邮件分类/文本分类等)(比如 jult --提示--> july)

4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?

Ans: 语言是千变万化、与时俱进的，使用正则匹配和语法无法满足现实需求或者是无法投入商业使用，而且后两者维护起来相对麻烦，对出现的新的语言或语句没有很好的处理方式

5. What's the Language Model;

Ans: 语言模型基于概率模型，在大量语料的基础上通过概率模型判断某一语句发生的可能性

6. Can you came up with some sceneraies at which we could use Language Model?

Ans: 阿里/JD智能客服、语音识别、机器翻译

7. What's the 1-gram language model;

Ans: 一般在处理语言相关数据或模型时，可能特征值比较多，这是假设第N个特征值的出现只与其本身存在关系

8. What's the disadvantages and advantages of 1-gram language model;

Ans: 1-gram 适用于特征之间相关性比较弱的模型，如果相关性很强，那么1-gram的假设条件就会出现不比较大的误差

9. What't the 2-gram models;
Ans: 2-gram是在多个特征值出现的情况下，当前特征n出现的概率至于其下一个特征n+1有关，与其他的特征值无关



In [3]:
import random
import numpy as np
import pandas as pd
import re
import jieba
from collections import Counter

In [4]:
grammer_rules = """
host = 主演 电影 评论
主演 = 吴京 adj | 王宝强 adj | 周润发 adj
adj = 的 | 这部 | 这个 | 这辆
电影 = 杀破狼 | 战狼 | 唐人街探案 | 赌神 |
评论 = 太好了 | 垃圾 | 恶心 | 好吃 | 车 adj | 稳住 | 不错
"""

In [5]:
def create_grammer(gram, line_split='\n', word_split='='):
    grammer = {}
    for line in gram.split(line_split):
        if not line: continue
        key, value = line.split(word_split)
        grammer[''.join(key.split())] = [val.split() for val in value.split('|')]
    return grammer

In [6]:
def generate(sentence, target):
    grammer = create_grammer(sentence)
    if target not in grammer: 
        return target
    sen = [generate(sentence, t) for t in random.choice(grammer[target])]
    return ''.join(sen)

In [7]:
def generate_n(sentence, target, n):
    for i in range(n):
        yield generate(sentence, target)

In [8]:
def deal_txt(string):
    return re.findall('\w+', string)

In [9]:
def my_cut(string):
    return list(jieba.cut(string))

In [10]:
def word_gram(path='word_gram.txt'):
    with open(path, 'r') as f:
        word_gram = [line.strip('\n') for line in f.readlines() if line]
    return Counter(word_gram)
word_gram = word_gram()

In [11]:
def word_2_gram(path='word_2_gram.txt'):
    with open(path, 'r') as f:
        word_2_gram = [line.strip('\n') for line in f.readlines() if line]
    return Counter(word_2_gram)
word_2_gram = word_2_gram()

In [12]:
# 2-gram p(w1|w2)P(w2|w3)
def get_sentence_prod(sentence):
    sent = my_cut(sentence)
    prod = 1
    for i, value in enumerate(sent[:-2]):
        p = (word_2_gram[sent[i] + sent[i+1]]) / (word_gram[sent[i+1]] + 1)
        prod *= p if p else 1/len(word_gram) # 如果 p概率为0 用 1/len(word_gram)代替 是否合适？ 
    return (sentence, prod)

In [13]:
def generate_best(n=5, comments=None, grammer=None, host=None):
    """
    n: 生成句子数
    comments: 评论列表
    grammer：自定义语法
    """
    result = []
    if comments:
        for comm in comments:
            result = [get_sentence_prod(comm) for comm in comments if comm]
    else:
        for i in generate_n(grammer, host, n):
            result.append(get_sentence_prod(i))
    return sorted(result, key=lambda x: x[1], reverse=True)

In [14]:
best_comment = generate_best(grammer=grammer_rules, host='host')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/nx/swsc48jj13gd4_bzw8vcr8lc0000gn/T/jieba.cache
Loading model cost 1.117 seconds.
Prefix dict has been built succesfully.


In [15]:
best_comment

[('王宝强的恶心', 9.139013534879045e-05),
 ('王宝强的赌神稳住', 2.32347801734213e-05),
 ('吴京的杀破狼稳住', 9.323790990295336e-09),
 ('王宝强这部唐人街探案稳住', 8.325707705403082e-12),
 ('周润发这部杀破狼恶心', 1.5495454504751688e-12)]

In [21]:
generate_best(comments=['吴京战狼两部曲搞的很不错', '还行吧一般般'])

[('还行吧一般般', 0.02242277633048009), ('吴京战狼两部曲搞的很不错', 1.8925943792974417e-26)]